In [209]:
import numpy as np
import pandas as pd
import random

Lets start creating attributes and populating them according to the following Schema.

![](DMD_DatabaseSchema.png)


In [210]:
# Defining the number of samples to be generated for each attribute.
n = 1000

# Seed for result replication.
seed = 10
np.random.seed(seed)
random.seed(seed)

In [211]:
# First attribute needs to be the ID of every machine for specific identification.
digits = 8
min_value = 10**(digits - 1)
max_value = 10**digits - 1
machine_id = np.random.randint(min_value, max_value, size=[n])

In [212]:
# Adding the ID variables into a dataframe to construct a table.
Machine_df = pd.DataFrame(machine_id, columns=['Machine_ID'])
Machine_df['Machine_Type'] = random.choices(['Free', 'Taxed'], k=n)
Machine_df

,Machine_ID,Machine_Type
0,85209853,Taxed
1,99128932,Free
2,47143872,Taxed
3,33686513,Free
4,93371164,Taxed
...,...,...
995,17863336,Taxed
996,67603892,Taxed
997,67084577,Free
998,57963776,Taxed


In [213]:
# Checking the count of random instances created.
count = Machine_df.groupby('Machine_Type').size()
count

Machine_Type
Free     488
Taxed    512
dtype: int64

In [214]:
# Lets create the company ID as an alphanumeric string for which we need string library.
import string
random.seed(seed)

n = 15
digits = 6
min_value = 10**(digits - 1)
max_value = 10**digits - 1


def alphanumeric_gen(digits):
    alphanumeric = string.ascii_uppercase + string.digits
    return ''.join(random.choice(alphanumeric) for _ in range(digits))

def entry_gen(n, digits):
    return [alphanumeric_gen(digits) for _ in range(n)]

company_ID = entry_gen(n, digits)

company_df = pd.DataFrame(company_ID, columns=['Company_ID'])
company_names = ["Sainsbury's", "Peri Peri", "Pizzahut", "Dominos", "Tasty's", "Chaiiwala", "NEXT", "Matalan", "Primark", "Tesco", "Co-op", "Mark & Spencer's", "Superdrug", "Starbucks", "KFC"]
company_df['Name'] = company_names

company_df.head()

,Company_ID,Name
0,C14AN3,Sainsbury's
1,5RKC75,Peri Peri
2,UEPXC0,Pizzahut
3,IWY0SQ,Dominos
4,3LTXI3,Tasty's


In [215]:
# Lets add the company ID to the Machine DF.
random.seed(seed)

# Assiging random company IDs to the machines in to act as foreign key.
def cid_rc(row):
    random_company = random.choice(company_df['Company_ID'].to_list())
    return random_company


Machine_df['Company_ID'] = Machine_df.apply(cid_rc, axis=1)
Machine_df

,Machine_ID,Machine_Type,Company_ID
0,85209853,Taxed,CC5TE8
1,99128932,Free,C14AN3
2,47143872,Taxed,IMT8XP
3,33686513,Free,U9214E
4,93371164,Taxed,CC5TE8
...,...,...,...
995,17863336,Taxed,FJYXJH
996,67603892,Taxed,G2KMW1
997,67084577,Free,LHR3TK
998,57963776,Taxed,U9214E


In [216]:
# Lets generate british format postal codes for the Locations table.
random.seed(seed)


def postal_gen():
    # Hard coding the prefixes to make the data realistic.
    prefixes = {
        'London': ['SW', 'SE', 'W', 'NW', 'N', 'E', 'WC', 'EC'],
        'Surrey': ['GU', 'KT', 'RH'],
        'Manchester':['M'],
        'Birmingham':['B'],
        'Edinburgh':['EH']
    }

    county = random.choice(list(prefixes.keys()))
    prefix = random.choice(prefixes[county])

    # Adding 2 numbers if the prefix is single letter.
    if len(prefix) == 1:
        second_part = ''.join(random.choices('01234', k=2))
    else:
        second_part = ''.join(random.choices('0123456789', k=1))

    third_part = ''.join(random.choices('0123456789', k=1))
    third_part += ''.join(random.choices('ABCDEFGHJKPSTUW', k=2))

    # To define area names based on the selected county from random choice.
    if county=='London':
        if prefix=='SW':
            name = 'South West London'
        elif prefix=='SE':
            name = 'South East London'
        elif prefix=='W':
            name = 'West London'
        elif prefix=='NW':
            name = 'North West London'
        elif prefix=='N':
            name = 'North London'
        elif prefix=='E':
            name = 'East London'
        elif prefix=='WC':
            name = 'West Central London'
        else:
            name = "East Central London"
    elif county=='Surrey':
        name = 'Surrey'
    elif county=='Manchester':
        name = 'Manchester'
    elif county=='Birmingham':
        name = 'Birmingham'
    else:
        name = 'Edinburgh'

    # Combining the three parts created above.
    postal_code = f"{prefix}{second_part} {third_part}"

    return postal_code, name

# Lets join the generated postal codes into a dataframe.
postal_codes = [postal_gen() for _ in range(50)]

location_df = pd.DataFrame(postal_codes, columns=['Postal_Code', 'Name'])

location_df.head()


,Postal_Code,Name
0,EH4 5DT,Edinburgh
1,M02 0WS,Manchester
2,M20 3GT,Manchester
3,M01 3GT,Manchester
4,KT6 0AC,Surrey


In [217]:
# Lets add these postal codes into the other tables randomly.
random.seed(seed)

# Assinging random postcodes to the company and Machine tables to act as foreign keys.
def pc_rc(row):
    random_pc = random.choice(location_df['Postal_Code'].to_list())
    return random_pc

company_df['Postal_Code'] = company_df.apply(pc_rc, axis=1)
Machine_df['Postal_Code'] = Machine_df.apply(pc_rc, axis=1)

In [218]:
# Lets create the final table i.e. Transactions.
np.random.seed(seed)
random.seed(seed)

n=5000

# Creating a normal distribution of data by centering around 150 for most transactions.
amount1 = np.random.normal(loc=150, scale=70, size=4900)
amount2 = np.random.uniform(300, 1000, size=100)
amount = np.concatenate((amount1, amount2))

# Rounding off and converting to a Dataframe.
transaction_df = pd.DataFrame(np.round(amount,2), columns=['Amount'])

# Assiging random machine IDs to transactions from Machine_df to act as foreign key.
def Mid_rc(row):
    random_Mid = random.choice(Machine_df['Machine_ID'].to_list())
    return random_Mid

transaction_df['Machine_ID'] = transaction_df.apply(Mid_rc, axis=1)

# Creating a categorical variable as transfer, purchase or withdrawal.
transaction_df['Transaction_Type'] = random.choices(['Transfer', 'Purchase', 'Withdrawal'], k=n)


In [219]:
# Lets generate random samples of date and time for Transaction date and time.
from datetime import datetime, timedelta
random.seed(seed)

# Generating random date and times for transactions.
def datetime_gen(startdate, enddate):
    delta = enddate - startdate
    days = random.randint(0, delta.days)
    seconds = random.randint(0, 24*60*60)
    return startdate + timedelta(days=days, seconds=seconds)

# Setting range for random sample generation.
startdate = datetime(2018, 1, 1)
enddate = datetime(2020, 12, 31)

transaction_date = [datetime_gen(startdate, enddate).strftime("%d-%m-%y %H:%M:%S") for _ in range(n)]

transaction_df['Transaction_date'] = transaction_date


In [220]:
# Adding a new column showing the tax amount for the transactions.
random.seed(seed)

def tax_gen(transaction_amount):
    tax_range = (0, 5)

    tax_rate = min(1, transaction_amount / 1000)

    return random.uniform(*tax_range) * tax_rate


def tax_update(row):
    machine_type = Machine_df.loc[Machine_df['Machine_ID'] == row['Machine_ID'],\
                                  'Machine_Type'].iloc[0]
    return 0 if machine_type == 'Free' else row['Tax']

# Generating a tax amount based on transaction amount
transaction_df['Tax'] = np.round(transaction_df['Amount'].apply(tax_gen), 2)

# Updating taxed amount based on machine type mentioned in the Machine DF column.
# Amount set to '0' for Free machines.
transaction_df['Tax'] = transaction_df.apply(tax_update, axis=1)

# Rearranging columns to make dataset clear.
transaction_df = transaction_df[['Transaction_date', 'Amount', 'Tax', 'Machine_ID', 'Transaction_Type']]
transaction_df

,Transaction_date,Amount,Tax,Machine_ID,Transaction_Type
0,08-03-18 15:36:55,243.21,0.69,27833222,Transfer
1,15-09-20 21:02:51,200.07,0.43,60044225,Transfer
2,31-01-18 07:30:13,41.82,0.12,97733707,Withdrawal
3,05-08-20 17:53:15,149.41,0.00,89705626,Transfer
4,23-07-19 23:47:31,193.49,0.00,35040650,Purchase
...,...,...,...,...,...
4995,18-02-18 05:53:13,706.22,0.00,38931288,Purchase
4996,27-08-18 02:09:19,999.96,3.16,42685287,Withdrawal
4997,31-10-20 09:09:18,384.89,0.00,47405775,Withdrawal
4998,19-02-18 08:29:08,890.65,0.00,72231106,Withdrawal


In [221]:
np.random.seed(seed)

# Lets create a percentage of null values to make data more realistic.
null_percentage = 0.20

def null_gen(df, pk_column, null_percentage):

    df_result = df.copy()

    num_null = int((df.size - df[pk_column].count()) * null_percentage)

    mask = np.zeros(df.shape, dtype=bool)
    mask[np.random.choice(df.shape[0],  num_null), 
         np.random.choice(df.shape[1], num_null)] = True
    
    pk_index = df.columns.get_loc(pk_column)
    mask[:, pk_index] = False


    df_result[mask] = np.nan

    return df_result

In [222]:
Machine_df = null_gen(Machine_df, 'Machine_ID', null_percentage)
Machine_df = Machine_df[['Machine_ID', 'Company_ID', 'Postal_Code', 'Machine_Type']]
Machine_df

,Machine_ID,Company_ID,Postal_Code,Machine_Type
0,85209853,CC5TE8,NaN,Taxed
1,99128932,C14AN3,KT6 0AC,Free
2,47143872,IMT8XP,NaN,Taxed
3,33686513,U9214E,W40 7CK,Free
4,93371164,CC5TE8,N02 3GS,NaN
...,...,...,...,...
995,17863336,FJYXJH,RH9 7AT,Taxed
996,67603892,G2KMW1,NaN,Taxed
997,67084577,LHR3TK,B40 8TF,Free
998,57963776,U9214E,NaN,NaN


In [223]:
company_df.head()

,Company_ID,Name,Postal_Code
0,C14AN3,Sainsbury's,EH8 6JA
1,5RKC75,Peri Peri,M20 3GT
2,UEPXC0,Pizzahut,GU7 7TF
3,IWY0SQ,Dominos,B02 3PU
4,3LTXI3,Tasty's,EH8 6JA


In [224]:
location_df.head()

,Postal_Code,Name
0,EH4 5DT,Edinburgh
1,M02 0WS,Manchester
2,M20 3GT,Manchester
3,M01 3GT,Manchester
4,KT6 0AC,Surrey


In [225]:
transaction_df = null_gen(transaction_df, 'Machine_ID', null_percentage)
transaction_df

,Transaction_date,Amount,Tax,Machine_ID,Transaction_Type
0,08-03-18 15:36:55,243.21,0.69,27833222,Transfer
1,15-09-20 21:02:51,200.07,NaN,60044225,Transfer
2,NaN,41.82,NaN,97733707,NaN
3,05-08-20 17:53:15,149.41,NaN,89705626,Transfer
4,23-07-19 23:47:31,193.49,0.00,35040650,Purchase
...,...,...,...,...,...
4995,18-02-18 05:53:13,706.22,0.00,38931288,Purchase
4996,27-08-18 02:09:19,999.96,3.16,42685287,Withdrawal
4997,31-10-20 09:09:18,384.89,0.00,47405775,NaN
4998,19-02-18 08:29:08,NaN,0.00,72231106,Withdrawal
